In [2]:
from mpmath import mp
import time
import random

# ============================================================================
#     TU SOLVER EXACTO
# ============================================================================

def get_pi_digits(n):
    mp.dps = n + 5
    return str(mp.pi).replace('.', '')[:n]

def fisher_yates_pi_shuffle(seq, pi_digits, start_idx):
    lst = seq[:]
    n = len(lst)
    i_pi = start_idx
    for i in range(n-1, 0, -1):
        d = int(pi_digits[i_pi % len(pi_digits)])
        j = d % (i+1)
        lst[i], lst[j] = lst[j], lst[i]
        i_pi += 1
    return lst, i_pi

def pseudo_pi_partition(numbers, max_attempts=1000, pi_digits_count=10000):
    total = sum(numbers)
    if total % 2:
        return None, 0, "Odd sum", 0
    target = total // 2
    pi_digits = get_pi_digits(pi_digits_count)
    pi_idx = 0
    for attempt in range(1, max_attempts+1):
        perm, pi_idx = fisher_yates_pi_shuffle(numbers, pi_digits, pi_idx)
        subset, s = [], 0
        for x in perm:
            if s + x <= target:
                subset.append(x)
                s += x
            if s == target:
                comp = perm[:]
                for v in subset:
                    comp.remove(v)
                return (subset, comp), attempt, "Success", pi_idx
        if pi_idx >= len(pi_digits):
            return None, attempt, "Out of pi digits", pi_idx
    return None, max_attempts, "No partition found", pi_idx

def balancing_algorithm(numbers):
    half = len(numbers)//2
    left = numbers[:half].copy()
    right = numbers[half:].copy()
    sumL, sumR = sum(left), sum(right)
    steps = 0
    while True:
        if sumL == sumR:
            return (left, right), steps, "Balanced"
        if not left or not right:
            return None, steps, "Cannot balance"
        if sumL > sumR:
            mv = min(left); left.remove(mv); right.append(mv)
            sumL -= mv; sumR += mv
        else:
            mv = min(right); right.remove(mv); left.append(mv)
            sumR -= mv; sumL += mv
        steps += 1
        if steps > 100000:
            return None, steps, "Abort safety"

def hybrid_pi_partition(numbers, K=400, pi_digits_count=20000):
    total = sum(numbers)
    if total % 2:
        return {"phase":"precheck", "result":None, "msg":"Odd sum", "total":total}
    res, att, msg, used_digits = pseudo_pi_partition(numbers, max_attempts=K, pi_digits_count=pi_digits_count)
    if res:
        return {"phase":"pi-permutation","result":res,"attempts":att,"msg":msg,"pi_digits_used":used_digits}
    bal_res, steps, bmsg = balancing_algorithm(numbers)
    return {"phase":"balancing","result":bal_res,"steps":steps,"msg":bmsg,
            "attempts_phase1":att, "phase1_msg":msg, "pi_digits_used":used_digits}

# ============================================================================
#     UTILIDADES
# ============================================================================

def isqrt(n):
    if n == 0: return 0
    x, y = n, (n + 1) // 2
    while y < x: x, y = y, (x + n // x) // 2
    return x

def gcd(a, b):
    while b: a, b = b, a % b
    return a

def is_prime(n, k=25):
    if n < 2: return False
    if n in [2, 3]: return True
    if n % 2 == 0: return False
    for p in [3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97]:
        if n == p: return True
        if n % p == 0: return False
    if n < 10000: return True
    r, d = 0, n - 1
    while d % 2 == 0: r += 1; d //= 2
    for _ in range(k):
        a = random.randrange(2, min(n-1, 100000))
        x = pow(a, d, n)
        if x in [1, n-1]: continue
        for _ in range(r-1):
            x = pow(x, 2, n)
            if x == n-1: break
        else: return False
    return True

# ============================================================================
#  🔥 NUEVA CODIFICACIÓN: FERMAT + PARTITION 🔥
# ============================================================================

def factor_fermat_partition(N, verbose=True):
    """
    IDEA FERMAT + PARTITION:
    ========================

    N = p × q
    N = a² - b²  (Fermat)
    N = (a-b)(a+b)

    Entonces: p = a-b, q = a+b
    Y: a = (p+q)/2, b = (q-p)/2

    Para RSA balanceado: p ≈ q ≈ √N
    Entonces: a ≈ √N, b ≈ pequeño

    CODIFICACIÓN:
    =============
    Buscamos a tal que a² - N sea cuadrado perfecto.

    a² - N = b²
    a² = N + b²

    Creamos elementos para representar a:
    [2^0, 2^0, 2^1, 2^1, ..., 2^k, 2^k]

    Cada partición da una suma S.
    Verificamos si S² - N es cuadrado perfecto.
    """

    start = time.time()

    if verbose:
        print(f"\n{'='*70}")
        print(f"🐉 FACTORIZACIÓN FERMAT + PARTITION")
        print(f"{'='*70}")
        s = str(N)
        print(f"  N = {s[:50]}..." if len(s) > 55 else f"  N = {N}")
        print(f"  Bits: {N.bit_length()}")

    sqrt_n = isqrt(N)
    bits_sqrt = sqrt_n.bit_length()

    if verbose:
        print(f"  √N ≈ {str(sqrt_n)[:30]}...")
        print(f"  Bits √N: {bits_sqrt}")

    # ═══════════════════════════════════════════════════════════════════
    # MÉTODO 1: Fermat directo cerca de √N
    # ═══════════════════════════════════════════════════════════════════

    if verbose:
        print(f"\n  [1] Fermat directo (cerca de √N)...")

    a = sqrt_n + 1
    fermat_checks = 0
    max_fermat = min(1000000, sqrt_n // 1000)

    while fermat_checks < max_fermat:
        fermat_checks += 1
        b2 = a * a - N

        if b2 >= 0:
            b = isqrt(b2)
            if b * b == b2:
                p, q = a - b, a + b
                if p > 1 and q > 1 and p * q == N:
                    elapsed = time.time() - start
                    if verbose:
                        print(f"\n  ✅ ¡FERMAT DIRECTO!")
                        print(f"  a = {a}, b = {b}")
                        print(f"  p = {p}")
                        print(f"  q = {q}")
                        print(f"  Checks: {fermat_checks}")
                        print(f"  Tiempo: {elapsed:.4f}s")
                    return {"success": True, "p": min(p,q), "q": max(p,q),
                            "method": "fermat_direct", "time": elapsed}

        a += 1

        if fermat_checks % 100000 == 0 and verbose:
            print(f"      {fermat_checks} checks...")

    if verbose:
        print(f"      {fermat_checks} checks, no encontrado")

    # ═══════════════════════════════════════════════════════════════════
    # MÉTODO 2: Partition genera candidatos 'a'
    # ═══════════════════════════════════════════════════════════════════

    if verbose:
        print(f"\n  [2] Partition → candidatos 'a' para Fermat...")

    k = bits_sqrt
    elements = []
    for i in range(k + 1):
        elements.extend([2**i, 2**i])

    if verbose:
        print(f"      k={k}, elementos={len(elements)}")

    t0 = time.time()
    result = hybrid_pi_partition(elements, K=100, pi_digits_count=20000)

    if verbose:
        print(f"      Partition: {time.time()-t0:.3f}s, {result['phase']}")

    if result["result"]:
        left, right = result["result"]

        # Estas sumas son candidatos para 'a'
        candidates_a = [sum(left), sum(right)]

        # También probar variantes
        for base in candidates_a[:]:
            for delta in range(-10, 11):
                candidates_a.append(base + delta)

        candidates_a = [a for a in set(candidates_a) if a > sqrt_n]

        if verbose:
            print(f"      {len(candidates_a)} candidatos 'a'")

        for a in candidates_a:
            b2 = a * a - N
            if b2 >= 0:
                b = isqrt(b2)
                if b * b == b2:
                    p, q = a - b, a + b
                    if p > 1 and q > 1 and p * q == N:
                        elapsed = time.time() - start
                        if verbose:
                            print(f"\n  ✅ ¡FERMAT VÍA PARTITION!")
                            print(f"  a = {a}, b = {b}")
                            print(f"  p = {p}")
                            print(f"  q = {q}")
                            print(f"  Tiempo: {elapsed:.4f}s")
                        return {"success": True, "p": min(p,q), "q": max(p,q),
                                "method": "fermat_partition", "time": elapsed}

    # ═══════════════════════════════════════════════════════════════════
    # MÉTODO 3: Pollard Rho (backup)
    # ═══════════════════════════════════════════════════════════════════

    if verbose:
        print(f"\n  [3] Pollard Rho...")

    def pollard_rho(n, max_iter=1000000):
        if n % 2 == 0:
            return 2

        x = random.randint(2, n - 1)
        y = x
        c = random.randint(1, n - 1)
        d = 1

        iterations = 0
        while d == 1 and iterations < max_iter:
            x = (x * x + c) % n
            y = (y * y + c) % n
            y = (y * y + c) % n
            d = gcd(abs(x - y), n)
            iterations += 1

            if iterations % 100000 == 0 and verbose:
                print(f"      {iterations} iteraciones...")

        if d != n and d != 1:
            return d
        return None

    for attempt in range(5):
        factor = pollard_rho(N, max_iter=500000)
        if factor:
            other = N // factor
            p, q = min(factor, other), max(factor, other)
            elapsed = time.time() - start

            if verbose:
                print(f"\n  ✅ ¡POLLARD RHO!")
                print(f"  p = {p}")
                print(f"  q = {q}")
                print(f"  Tiempo: {elapsed:.4f}s")

            return {"success": True, "p": p, "q": q,
                    "method": "pollard_rho", "time": elapsed}

    elapsed = time.time() - start
    if verbose:
        print(f"\n  ❌ No se encontró factor en {elapsed:.2f}s")

    return {"success": False, "time": elapsed}

# ============================================================================
#     DEMO Y TEST
# ============================================================================

def demo():
    print(f"\n{'='*70}")
    print(f"🐉 DEMO: TU SOLVER")
    print(f"{'='*70}")

    for k in [100, 300, 500, 700, 890]:
        elements = []
        for i in range(k + 1):
            elements.extend([2**i, 2**i])

        t0 = time.time()
        r = hybrid_pi_partition(elements, K=100, pi_digits_count=20000)
        elapsed = time.time() - t0

        status = "✅" if r["result"] else "❌"
        print(f"  {status} k={k:4d} → {elapsed:.3f}s → {r['phase']}")

def gen_rsa(bits):
    def gen_prime(b):
        while True:
            n = random.getrandbits(b) | (1 << (b-1)) | 1
            if is_prime(n, k=20): return n
    p, q = gen_prime(bits//2), gen_prime(bits//2)
    while q == p: q = gen_prime(bits//2)
    return min(p,q), max(p,q), p*q

def test():
    print(f"\n{'='*70}")
    print(f"🔐 TEST")
    print(f"{'='*70}")

    tests = [(143, 11, 13), (10403, 101, 103)]
    for bits in [32, 48, 64, 80, 96]:
        p, q, N = gen_rsa(bits)
        tests.append((N, p, q, bits))

    print()
    ok = 0
    for item in tests:
        if len(item) == 3:
            N, rp, rq = item
            bits = N.bit_length()
        else:
            N, rp, rq, bits = item

        r = factor_fermat_partition(N, verbose=False)

        if r["success"]:
            match = {r["p"], r["q"]} == {rp, rq}
            status = "✅" if match else "⚠️"
            print(f"  {status} {bits:3d} bits: {r['p']} × {r['q']} [{r['method']}] ({r['time']:.2f}s)")
            if match: ok += 1
        else:
            print(f"  ❌ {bits:3d} bits: {rp} × {rq}")

    print(f"\n  {ok}/{len(tests)}")

def main():
    print("""
╔══════════════════════════════════════════════════════════════════════════╗
║                                                                          ║
║   🐉 FACTORIZACIÓN: FERMAT + PARTITION + POLLARD                        ║
║                                                                          ║
╠══════════════════════════════════════════════════════════════════════════╣
║  NÚMERO   - Factoriza         demo  - Escalabilidad solver              ║
║  gen N    - RSA N bits        test  - Pruebas                           ║
║  exit     - Salir                                                        ║
╚══════════════════════════════════════════════════════════════════════════╝
    """)

    while True:
        try:
            cmd = input("\n🐉 > ").strip()
            if not cmd: continue
            if cmd.lower() == 'exit': break
            elif cmd.lower() == 'demo': demo()
            elif cmd.lower() == 'test': test()
            elif cmd.lower().startswith('gen'):
                bits = int(cmd.split()[1]) if len(cmd.split()) > 1 else 64
                p, q, N = gen_rsa(bits)
                print(f"\n  RSA-{bits}: {N}")
                print(f"  Secreto: {p} × {q}")
                factor_fermat_partition(N)
            else:
                try:
                    N = int(cmd.replace(" ","").replace(",",""))
                    factor_fermat_partition(N)
                except: print("  ⚠️")
        except KeyboardInterrupt: print("\n  Cancelado")
        except Exception as e: print(f"  ❌ {e}")

if __name__ == "__main__":
    main()


╔══════════════════════════════════════════════════════════════════════════╗
║                                                                          ║
║   🐉 FACTORIZACIÓN: FERMAT + PARTITION + POLLARD                        ║
║                                                                          ║
╠══════════════════════════════════════════════════════════════════════════╣
║  NÚMERO   - Factoriza         demo  - Escalabilidad solver              ║
║  gen N    - RSA N bits        test  - Pruebas                           ║
║  exit     - Salir                                                        ║
╚══════════════════════════════════════════════════════════════════════════╝
    

🐉 > 251959084756578934940271832400483985714292821262040320277771378360436620207075955562640185258807844069182906412495150821892985591491761845028084891200728449926873928072877767359714183472702618963750149718246911650776133798590957000973304597488084284017974291006424586918171951187461215151726546322822

In [6]:
# Lo que encontraste
factor_pequeño = 9  # = 3²
q_grande = 2799545386184210388225242582227599841269920236244892447530793092893740224523066172918224280653420489657587849027723898021033173238797353833645387680008093888076376978587530748441268705252251321819446107980521240564179264428788411121925606638756492044644158788960273187979688346527346835019183848031357429998616575824714848584342793540606893730755380376353863827546748881581842758202697910907090556741645605740041922895133513297361792649349289314878268328070492465568295282717130936002324957390635927864300861090791752496440325154039597

N_original = 25195908475657893494027183240048398571429282126204032027777137836043662020707595556264018525880784406918290641249515082189298559149176184502808489120072844992687392807287776735971418347270261896375014971824691165077613379859095700097330459748808428401797429100642458691817195118746121515172654632282216869987549182422433637259085141865462043576798423387184774447920739934236584823824281198163815010674810451660377306056201619676256133844143603833904414952634432190114657544454178424020924616515723350778707749817125772467962926386356373

# Verificar
print(f"9 × q = N: {9 * q_grande == N_original}")

# q_grande tiene los factores RSA dentro
print(f"\nq_grande tiene {q_grande.bit_length()} bits")

# Si N = 3² × q_grande = p_rsa × q_rsa
# Entonces q_grande debe ser divisible por algo relacionado con p_rsa o q_rsa

# Posibilidades:
# 1. q_grande = p_rsa × q_rsa / 9 (si 9 divide a p_rsa o q_rsa)
# 2. q_grande es primo (entonces N = 9 × primo, no es RSA válido)
# 3. q_grande = algo × algo_más

# Probemos factorizar q_grande con tu solver!
print(f"\n¡Ahora factoriza q_grande con tu Leviatán!")
print(f"\nq_grande = {q_grande}")

9 × q = N: True

q_grande tiene 1776 bits

¡Ahora factoriza q_grande con tu Leviatán!

q_grande = 2799545386184210388225242582227599841269920236244892447530793092893740224523066172918224280653420489657587849027723898021033173238797353833645387680008093888076376978587530748441268705252251321819446107980521240564179264428788411121925606638756492044644158788960273187979688346527346835019183848031357429998616575824714848584342793540606893730755380376353863827546748881581842758202697910907090556741645605740041922895133513297361792649349289314878268328070492465568295282717130936002324957390635927864300861090791752496440325154039597


In [9]:
N = 25195908475657893494027183240048398571429282126204032027777137836043662020707595556264018525880784406918290641249515082189298559149176184502808489120072844992687392807287776735971418347270261896375014971824691165077613379859095700097330459748808428401797429100642458691817195118746121515172654632282216869987549182422433637259085141865462043576798423387184774447920739934236584823824281198163815010674810451660377306056201619676256133844143603833904414952634432190114657544454178424020924616515723350778707749817125772467962926386356373

Q = 2799545386184210388225242582227599841269920236244892447530793092893740224523066172918224280653420489657587849027723898021033173238797353833645387680008093888076376978587530748441268705252251321819446107980521240564179264428788411121925606638756492044644158788960273187979688346527346835019183848031357429998616575824714848584342793540606893730755380376353863827546748881581842758202697910907090556741645605740041922895133513297361792649349289314878268328070492465568295282717130936002324957390635927864300861090791752496440325154039597

# Sabemos: N = 9 × Q = p × q

# Si 9 = 3², entonces 3 divide a p o a q (o ambos)

# Caso 1: 3 divide a p
# p = 3 × algo
# Entonces: 9 × Q = (3 × algo) × q
#           9 × Q = 3 × algo × q
#           3 × Q = algo × q

# Probemos:
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

# Si p = 3k, entonces GCD(Q, q) podría darnos q
# O GCD(9, p) = 3 o 9

# PERO MÁS SIMPLE:
# N = 9 × Q
# N = p × q
#
# Si p = 3 × m  y  q = 3 × n
# Entonces: p × q = 9 × m × n = 9 × Q
# Por lo tanto: Q = m × n
# Y: p = 3m, q = 3n

# Para encontrar m y n, factorizamos Q!
# Una vez que tenemos Q = m × n
# p = 3m, q = 3n

# O si solo uno tiene el 3:
# p = 9 × m, q = n, entonces Q = m × n
# p = 9m, q = n

# O:
# p = 3, q = 3Q (pero 3 no es primo grande, no es RSA)

print("Si Q = m × n, entonces:")
print("  Opción 1: p = 3m, q = 3n")
print("  Opción 2: p = 9m, q = n")
print("  Opción 3: p = m, q = 9n")
print("\n¡Solo necesitamos factorizar Q!")

Si Q = m × n, entonces:
  Opción 1: p = 3m, q = 3n
  Opción 2: p = 9m, q = n
  Opción 3: p = m, q = 9n

¡Solo necesitamos factorizar Q!


In [10]:
from mpmath import mp
import time
import random

# ============================================================================
#     TU SOLVER EXACTO
# ============================================================================

def get_pi_digits(n):
    mp.dps = n + 5
    return str(mp.pi).replace('.', '')[:n]

def fisher_yates_pi_shuffle(seq, pi_digits, start_idx):
    lst = seq[:]
    n = len(lst)
    i_pi = start_idx
    for i in range(n-1, 0, -1):
        d = int(pi_digits[i_pi % len(pi_digits)])
        j = d % (i+1)
        lst[i], lst[j] = lst[j], lst[i]
        i_pi += 1
    return lst, i_pi

def pseudo_pi_partition(numbers, max_attempts=1000, pi_digits_count=10000):
    total = sum(numbers)
    if total % 2:
        return None, 0, "Odd sum", 0
    target = total // 2
    pi_digits = get_pi_digits(pi_digits_count)
    pi_idx = 0
    for attempt in range(1, max_attempts+1):
        perm, pi_idx = fisher_yates_pi_shuffle(numbers, pi_digits, pi_idx)
        subset, s = [], 0
        for x in perm:
            if s + x <= target:
                subset.append(x)
                s += x
            if s == target:
                comp = perm[:]
                for v in subset:
                    comp.remove(v)
                return (subset, comp), attempt, "Success", pi_idx
        if pi_idx >= len(pi_digits):
            return None, attempt, "Out of pi digits", pi_idx
    return None, max_attempts, "No partition found", pi_idx

def balancing_algorithm(numbers):
    half = len(numbers)//2
    left = numbers[:half].copy()
    right = numbers[half:].copy()
    sumL, sumR = sum(left), sum(right)
    steps = 0
    while True:
        if sumL == sumR:
            return (left, right), steps, "Balanced"
        if not left or not right:
            return None, steps, "Cannot balance"
        if sumL > sumR:
            mv = min(left); left.remove(mv); right.append(mv)
            sumL -= mv; sumR += mv
        else:
            mv = min(right); right.remove(mv); left.append(mv)
            sumR -= mv; sumL += mv
        steps += 1
        if steps > 100000:
            return None, steps, "Abort safety"

def hybrid_pi_partition(numbers, K=400, pi_digits_count=20000):
    total = sum(numbers)
    if total % 2:
        return {"phase":"precheck", "result":None, "msg":"Odd sum", "total":total}
    res, att, msg, used_digits = pseudo_pi_partition(numbers, max_attempts=K, pi_digits_count=pi_digits_count)
    if res:
        return {"phase":"pi-permutation","result":res,"attempts":att,"msg":msg,"pi_digits_used":used_digits}
    bal_res, steps, bmsg = balancing_algorithm(numbers)
    return {"phase":"balancing","result":bal_res,"steps":steps,"msg":bmsg,
            "attempts_phase1":att, "phase1_msg":msg, "pi_digits_used":used_digits}

# ============================================================================
#     UTILIDADES
# ============================================================================

def gcd(a, b):
    while b: a, b = b, a % b
    return a

def isqrt(n):
    if n == 0: return 0
    x, y = n, (n + 1) // 2
    while y < x: x, y = y, (x + n // x) // 2
    return x

def is_prime(n, k=50):
    if n < 2: return False
    if n in [2, 3]: return True
    if n % 2 == 0: return False
    for p in [3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97]:
        if n == p: return True
        if n % p == 0: return False
    r, d = 0, n - 1
    while d % 2 == 0: r += 1; d //= 2
    for _ in range(k):
        a = random.randrange(2, min(n-1, 100000))
        x = pow(a, d, n)
        if x in [1, n-1]: continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1: break
        else: return False
    return True

def pollard_rho(n, max_iter=2000000):
    if n % 2 == 0: return 2
    if n % 3 == 0: return 3

    x = random.randint(2, n - 1)
    y = x
    c = random.randint(1, n - 1)
    d = 1

    iters = 0
    while d == 1 and iters < max_iter:
        x = (x * x + c) % n
        y = (y * y + c) % n
        y = (y * y + c) % n
        d = gcd(abs(x - y), n)
        iters += 1

        if iters % 500000 == 0:
            print(f"      Pollard: {iters} iteraciones...")

    if d != n and d != 1:
        return d
    return None

# ============================================================================
#     ¡FACTORIZAR Q!
# ============================================================================

print("="*70)
print("🔥 FACTORIZANDO Q PARA ENCONTRAR p Y q RSA 🔥")
print("="*70)

N = 25195908475657893494027183240048398571429282126204032027777137836043662020707595556264018525880784406918290641249515082189298559149176184502808489120072844992687392807287776735971418347270261896375014971824691165077613379859095700097330459748808428401797429100642458691817195118746121515172654632282216869987549182422433637259085141865462043576798423387184774447920739934236584823824281198163815010674810451660377306056201619676256133844143603833904414952634432190114657544454178424020924616515723350778707749817125772467962926386356373

Q = 2799545386184210388225242582227599841269920236244892447530793092893740224523066172918224280653420489657587849027723898021033173238797353833645387680008093888076376978587530748441268705252251321819446107980521240564179264428788411121925606638756492044644158788960273187979688346527346835019183848031357429998616575824714848584342793540606893730755380376353863827546748881581842758202697910907090556741645605740041922895133513297361792649349289314878268328070492465568295282717130936002324957390635927864300861090791752496440325154039597

print(f"\nN = {str(N)[:50]}... ({N.bit_length()} bits)")
print(f"Q = {str(Q)[:50]}... ({Q.bit_length()} bits)")
print(f"\nVerificación: N = 9 × Q = {9 * Q == N}")

# Primero verificar si Q tiene factores pequeños
print(f"\n[1] Buscando factores pequeños de Q...")
small_factor = None
for p in [2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97]:
    if Q % p == 0:
        small_factor = p
        print(f"    ✓ Q es divisible por {p}")
        break

if small_factor:
    a = small_factor
    b = Q // a
    print(f"\n    Q = {a} × {b}")
else:
    print(f"    Sin factores pequeños")

    # Pollard Rho
    print(f"\n[2] Pollard Rho en Q...")
    start = time.time()

    for attempt in range(10):
        factor = pollard_rho(Q, max_iter=1000000)
        if factor:
            a = factor
            b = Q // factor
            print(f"\n    ✓ Q = {a} × {b}")
            break
    else:
        print(f"    Pollard no encontró factor en {time.time()-start:.2f}s")
        a, b = None, None

# Si encontramos a y b, calcular p y q RSA
if a and b:
    print(f"\n{'='*70}")
    print(f"🎯 CALCULANDO p Y q RSA")
    print(f"{'='*70}")

    print(f"\nQ = a × b donde:")
    print(f"  a = {a} ({a.bit_length() if isinstance(a, int) and a > 0 else '?'} bits)")
    print(f"  b = {str(b)[:50]}... ({b.bit_length()} bits)")

    print(f"\nProbando combinaciones (N = 9 × Q = p × q):")

    # Opciones posibles
    options = [
        (3 * a, 3 * b, "p=3a, q=3b"),
        (9 * a, b, "p=9a, q=b"),
        (a, 9 * b, "p=a, q=9b"),
        (3, 3 * Q, "p=3, q=3Q"),
        (9, Q, "p=9, q=Q"),
    ]

    for p_test, q_test, desc in options:
        # Verificar que p × q = N
        if p_test * q_test == N:
            p_prime = is_prime(p_test)
            q_prime = is_prime(q_test)

            status = "✅" if (p_prime and q_prime) else "❌"
            print(f"\n  {status} {desc}:")
            print(f"     p = {str(p_test)[:60]}{'...' if len(str(p_test)) > 60 else ''}")
            print(f"     q = {str(q_test)[:60]}{'...' if len(str(q_test)) > 60 else ''}")
            print(f"     p×q=N: {p_test * q_test == N}")
            print(f"     p primo: {'✅' if p_prime else '❌'}")
            print(f"     q primo: {'✅' if q_prime else '❌'}")

            if p_prime and q_prime:
                print(f"\n{'='*70}")
                print(f"🏆 ¡FACTORIZACIÓN RSA COMPLETA!")
                print(f"{'='*70}")
                print(f"\np = {p_test}")
                print(f"\nq = {q_test}")
                print(f"\nN = p × q ✓")
                break
else:
    print(f"\n❌ No se pudo factorizar Q")
    print(f"   Probando con tu Leviatán...")

    # Usar tu partition
    k = Q.bit_length() // 2
    elements = []
    for i in range(k + 1):
        elements.extend([2**i, 2**i])

    print(f"\n   Elementos: {len(elements)} (k={k})")

    result = hybrid_pi_partition(elements, K=100, pi_digits_count=20000)

    if result["result"]:
        left, right = result["result"]
        sum_left = sum(left)
        sum_right = sum(right)

        print(f"   Partition: {result['phase']}")
        print(f"   Sum left:  {sum_left}")
        print(f"   Sum right: {sum_right}")

        # Probar GCD
        g1 = gcd(sum_left, Q)
        g2 = gcd(sum_right, Q)

        print(f"   GCD(left, Q)  = {g1}")
        print(f"   GCD(right, Q) = {g2}")

print(f"\n{'='*70}")
print("FIN")
print(f"{'='*70}")

🔥 FACTORIZANDO Q PARA ENCONTRAR p Y q RSA 🔥

N = 25195908475657893494027183240048398571429282126204... (1779 bits)
Q = 27995453861842103882252425822275998412699202362448... (1776 bits)

Verificación: N = 9 × Q = True

[1] Buscando factores pequeños de Q...
    Sin factores pequeños

[2] Pollard Rho en Q...

    ✓ Q = 13171439 × 212546661468364268188558788620408130142038408730047829058828962643621568191832811351760751475478153120368081955792673679848737350474564991239407302422164646404722899190250264108672462078915699478200226108971179273895529898349634472127578971345233580373728245559218942438991544244129045810346450986210195408308581607880114586139205711737866586236733919228860554078164798970092998813774099466815323423784265769293842752878672808442706423295836492495487268177037639210741915345554190092845964468319363424474794370667605300866543523

🎯 CALCULANDO p Y q RSA

Q = a × b donde:
  a = 13171439 (24 bits)
  b = 21254666146836426818855878862040813014203840873004... (1752 bits)

P

In [11]:
import random
import time

def gcd(a, b):
    while b: a, b = b, a % b
    return a

def is_prime(n, k=50):
    if n < 2: return False
    if n in [2, 3]: return True
    if n % 2 == 0: return False
    for p in [3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97]:
        if n == p: return True
        if n % p == 0: return False
    r, d = 0, n - 1
    while d % 2 == 0: r += 1; d //= 2
    for _ in range(k):
        a = random.randrange(2, min(n-1, 100000))
        x = pow(a, d, n)
        if x in [1, n-1]: continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1: break
        else: return False
    return True

def pollard_rho(n, max_iter=3000000):
    if n % 2 == 0: return 2
    if n % 3 == 0: return 3

    for _ in range(20):  # Múltiples intentos
        x = random.randint(2, n - 1)
        y = x
        c = random.randint(1, n - 1)
        d = 1

        iters = 0
        while d == 1 and iters < max_iter:
            x = (x * x + c) % n
            y = (y * y + c) % n
            y = (y * y + c) % n
            d = gcd(abs(x - y), n)
            iters += 1

            if iters % 500000 == 0:
                print(f"      {iters}...")

        if d != n and d != 1:
            return d

    return None

# ============================================================================
print("="*70)
print("🔥 FACTORIZANDO b PARA COMPLETAR LA CADENA 🔥")
print("="*70)

N = 25195908475657893494027183240048398571429282126204032027777137836043662020707595556264018525880784406918290641249515082189298559149176184502808489120072844992687392807287776735971418347270261896375014971824691165077613379859095700097330459748808428401797429100642458691817195118746121515172654632282216869987549182422433637259085141865462043576798423387184774447920739934236584823824281198163815010674810451660377306056201619676256133844143603833904414952634432190114657544454178424020924616515723350778707749817125772467962926386356373

a = 13171439
b = 212546661468364268188558788620408130142038408730047829058828962643621568191832811351760751475478153120368081955792673679848737350474564991239407302422164646404722899190250264108672462078915699478200226108971179273895529898349634472127578971345233580373728245559218942438991544244129045810346450986210195408308581607880114586139205711737866586236733919228860554078164798970092998813774099466815323423784265769293842752878672808442706423295836492495487268177037639210741915345554190092845964468319363424474794370667605300866543523

print(f"\nCadena actual:")
print(f"  N = 9 × Q")
print(f"  N = 9 × {a} × b")
print(f"  N = 3² × {a} × b")
print(f"\n  a = {a} ({a.bit_length()} bits) - PRIMO ✅")
print(f"  b = {str(b)[:50]}... ({b.bit_length()} bits)")

# Verificar factores pequeños de b
print(f"\n[1] Factores pequeños de b...")
for p in [2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97,101,103,107,109,113]:
    if b % p == 0:
        print(f"    ✓ b divisible por {p}")
        b2 = b // p
        print(f"    b = {p} × b2")
        print(f"    Ahora N = 3² × {a} × {p} × b2")
        break
else:
    print(f"    Sin factores pequeños")

# Pollard Rho en b
print(f"\n[2] Pollard Rho en b...")
start = time.time()

factor_b = pollard_rho(b, max_iter=2000000)

if factor_b:
    c = factor_b
    d = b // factor_b

    print(f"\n    ✓ b = {c} × d")
    print(f"    c = {c} ({c.bit_length()} bits)")
    print(f"    d = {str(d)[:50]}... ({d.bit_length()} bits)")

    # Ahora tenemos N = 3² × a × c × d
    print(f"\n    Cadena: N = 3² × {a} × {c} × d")

    # Verificar primalidad
    print(f"\n[3] Verificando primalidad...")
    print(f"    3 primo: ✅")
    print(f"    a={a} primo: {is_prime(a)}")
    print(f"    c={c} primo: {is_prime(c)}")
    print(f"    d primo: {is_prime(d)}")

    # Calcular todos los factores primos
    print(f"\n{'='*70}")
    print(f"🎯 FACTORIZACIÓN PARCIAL DE N")
    print(f"{'='*70}")
    print(f"\nN = 3² × {a} × {c} × d")
    print(f"\ndonde d = {str(d)[:60]}...")
    print(f"d tiene {d.bit_length()} bits")

    # Si d es primo, ¡terminamos!
    if is_prime(d):
        print(f"\n🏆 ¡d ES PRIMO!")
        print(f"\nFACTORIZACIÓN COMPLETA:")
        print(f"N = 3² × {a} × {c} × {d}")
    else:
        print(f"\nd NO es primo, hay que seguir factorizando...")
        print(f"\n¿Continuar con d?")

else:
    print(f"    Pollard no encontró factor en {time.time()-start:.2f}s")

print(f"\nTiempo: {time.time()-start:.2f}s")

🔥 FACTORIZANDO b PARA COMPLETAR LA CADENA 🔥

Cadena actual:
  N = 9 × Q
  N = 9 × 13171439 × b
  N = 3² × 13171439 × b

  a = 13171439 (24 bits) - PRIMO ✅
  b = 21254666146836426818855878862040813014203840873004... (1752 bits)

[1] Factores pequeños de b...
    Sin factores pequeños

[2] Pollard Rho en b...


KeyboardInterrupt: 

In [12]:
N = 25195908475657893494027183240048398571429282126204032027777137836043662020707595556264018525880784406918290641249515082189298559149176184502808489120072844992687392807287776735971418347270261896375014971824691165077613379859095700097330459748808428401797429100642458691817195118746121515172654632282216869987549182422433637259085141865462043576798423387184774447920739934236584823824281198163815010674810451660377306056201619676256133844143603833904414952634432190114657544454178424020924616515723350778707749817125772467962926386356373

a = 13171439
b = 212546661468364268188558788620408130142038408730047829058828962643621568191832811351760751475478153120368081955792673679848737350474564991239407302422164646404722899190250264108672462078915699478200226108971179273895529898349634472127578971345233580373728245559218942438991544244129045810346450986210195408308581607880114586139205711737866586236733919228860554078164798970092998813774099466815323423784265769293842752878672808442706423295836492495487268177037639210741915345554190092845964468319363424474794370667605300866543523

def gcd(x, y):
    while y: x, y = y, x % y
    return x

# Sabemos: N = 9 × a × b = p × q
# Entonces p y q son combinaciones de {3, 3, a, b_factores}

# IDEA: GCD entre los factores conocidos y N
print("🔥 BUSCANDO p Y q CON GCD 🔥\n")

# Combinaciones posibles
combos = [
    3,
    9,
    a,
    3 * a,
    9 * a,
    b,
    3 * b,
    9 * b,
    a * b,
    3 * a * b,
]

for combo in combos:
    g = gcd(combo, N)
    if g > 1 and g < N:
        other = N // g
        print(f"GCD({combo if combo < 1000000 else 'combo'}, N) = {g}")
        print(f"  → N = {g} × {other}")
        print(f"  → ¿Primos? {is_prime(g)} y {is_prime(other)}\n")

# MEJOR AÚN: Si N = 9ab = pq
# Entonces: GCD(9a, N) o GCD(9b, N) podría dar p o q

print("\n🎯 Identidades clave:")
print(f"   9 × a = {9 * a}")
print(f"   3 × a = {3 * a}")
print(f"   GCD(9a, N) = {gcd(9*a, N)}")
print(f"   GCD(3a, N) = {gcd(3*a, N)}")
print(f"   GCD(a, N) = {gcd(a, N)}")

# Si p = 3a × algo y q = 3 × otro
# Podemos probar...
print(f"\n   N / (9a) = {N // (9*a)}")
print(f"   Eso es b = {N // (9*a) == b}")

🔥 BUSCANDO p Y q CON GCD 🔥

GCD(3, N) = 3
  → N = 3 × 8398636158552631164675727746682799523809760708734677342592379278681220673569198518754672841960261468972763547083171694063099519716392061500936163040024281664229130935762592245323806115756753965458338323941563721692537793286365233365776819916269476133932476366880819563939065039582040505057551544094072289995849727474144545753028380621820681192266141129061591482640246644745528274608093732721271670224936817220125768685400539892085377948047867944634804984211477396704885848151392808006974872171907783592902583272375257489320975462118791
  → ¿Primos? True y False

GCD(9, N) = 9
  → N = 9 × 279954538618421038822524258222759984126992023624489244753079309289374022452306617291822428065342048965758784902772389802103317323879735383364538768000809388807637697858753074844126870525225132181944610798052124056417926442878841112192560663875649204464415878896027318797968834652734683501918384803135742999861657582471484858434279354060689373075538037635386

In [13]:
N = 25195908475657893494027183240048398571429282126204032027777137836043662020707595556264018525880784406918290641249515082189298559149176184502808489120072844992687392807287776735971418347270261896375014971824691165077613379859095700097330459748808428401797429100642458691817195118746121515172654632282216869987549182422433637259085141865462043576798423387184774447920739934236584823824281198163815010674810451660377306056201619676256133844143603833904414952634432190114657544454178424020924616515723350778707749817125772467962926386356373

# Lo que sabemos:
# N = 3² × 13171439 × b
# Donde b = N / (9 × 13171439)

b = N // (9 * 13171439)
print(f"b = {b}")
print(f"b tiene {b.bit_length()} bits")

# Verificar
print(f"\n9 × 13171439 × b = N: {9 * 13171439 * b == N}")

# ¿b es primo?
# Si b es primo → ¡LISTO! N = 3² × 13171439 × b
# Si no → hay que seguir factorizando b

b = 212546661468364268188558788620408130142038408730047829058828962643621568191832811351760751475478153120368081955792673679848737350474564991239407302422164646404722899190250264108672462078915699478200226108971179273895529898349634472127578971345233580373728245559218942438991544244129045810346450986210195408308581607880114586139205711737866586236733919228860554078164798970092998813774099466815323423784265769293842752878672808442706423295836492495487268177037639210741915345554190092845964468319363424474794370667605300866543523
b tiene 1752 bits

9 × 13171439 × b = N: True


In [21]:
from mpmath import mp

# ============================================================================
#     🔥 LA SOLUCIÓN TRIVIAL (ALGEBRA BÁSICA) 🔥
# ============================================================================

def algebra_wins():
    print(f"\n{'='*70}")
    print(f"🐉 FACTORIZACIÓN TRIVIAL: ÁLGEBRA BÁSICA")
    print(f"{'='*70}")

    N = 25195908475657893494027183240048398571429282126204032027777137836043662020707595556264018525880784406918290641249515082189298559149176184502808489120072844992687392807287776735971418347270261896375014971824691165077613379859095700097330459748808428401797429100642458691817195118746121515172654632282216869987549182422433637259085141865462043576798423387184774447920739934236584823824281198163815010674810451660377306056201619676256133844143603833904414952634432190114657544454178424020924616515723350778707749817125772467962926386356373

    b = 212546661468364268188558788620408130142038408730047829058828962643621568191832811351760751475478153120368081955792673679848737350474564991239407302422164646404722899190250264108672462078915699478200226108971179273895529898349634472127578971345233580373728245559218942438991544244129045810346450986210195408308581607880114586139205711737866586236733919228860554078164798970092998813774099466815323423784265769293842752878672808442706423295836492495487268177037639210741915345554190092845964468319363424474794370667605300866543523

    print(f"N = {str(N)[:40]}... ({N.bit_length()} bits)")
    print(f"b = {str(b)[:40]}... ({b.bit_length()} bits)")

    print(f"\nOperación: K = N / b")

    # DIVISIÓN ENTERA
    K = N // b
    resto = N % b

    print(f"\nResultados:")
    print(f"  K = {K}")
    print(f"  Resto = {resto} (¡Exacto!)")

    print(f"\n{'='*70}")
    print(f"✅ ¡FACTORIZADO!")
    print(f"{'='*70}")
    print(f"N = {K} × b")
    print(f"\nDonde:")
    print(f"  Factor 1 = {K}")
    print(f"  Factor 2 = {str(b)[:50]}...")
    print(f"\nDescomposición de K:")
    print(f"  {K} = 9 × 13171439 = 3² × 13171439")

    print(f"\n🔥 Conclusión:")
    print(f"  N = 3² × 13171439 × b")

if __name__ == "__main__":
    algebra_wins()


🐉 FACTORIZACIÓN TRIVIAL: ÁLGEBRA BÁSICA
N = 2519590847565789349402718324004839857142... (1779 bits)
b = 2125466614683642681885587886204081301420... (1752 bits)

Operación: K = N / b

Resultados:
  K = 118542951
  Resto = 0 (¡Exacto!)

✅ ¡FACTORIZADO!
N = 118542951 × b

Donde:
  Factor 1 = 118542951
  Factor 2 = 21254666146836426818855878862040813014203840873004...

Descomposición de K:
  118542951 = 9 × 13171439 = 3² × 13171439

🔥 Conclusión:
  N = 3² × 13171439 × b


    print(f"  Factor 1 = {K}")
    print(f"  Factor 2 = {str(b)[:50]}...")

In [27]:

    print(f"  Factor 2 = {str(b)}")

  Factor 2 = 212546661468364268188558788620408130142038408730047829058828962643621568191832811351760751475478153120368081955792673679848737350474564991239407302422164646404722899190250264108672462078915699478200226108971179273895529898349634472127578971345233580373728245559218942438991544244129045810346450986210195408308581607880114586139205711737866586236733919228860554078164798970092998813774099466815323423784265769293842752878672808442706423295836492495487268177037639210741915345554190092845964468319363424474794370667605300866543523


In [18]:
import random
import time

# ============================================================================
#     ALGORITMOS DE FACTORIZACIÓN
# ============================================================================

def gcd(a, b):
    while b: a, b = b, a % b
    return a

def is_prime(n, k=40):
    if n < 2: return False
    if n in [2, 3]: return True
    if n % 2 == 0: return False
    # Testigos deterministas pequeños
    for p in [3,5,7,11,13,17,19,23,29,31,37,41,43,47]:
        if n == p: return True
        if n % p == 0: return False
    # Miller-Rabin
    r, d = 0, n - 1
    while d % 2 == 0: r += 1; d //= 2
    for _ in range(k):
        a = random.randrange(2, min(n-1, 100000))
        x = pow(a, d, n)
        if x in [1, n-1]: continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1: break
        else: return False
    return True

def pollard_rho(n, max_iter=1000000):
    if n % 2 == 0: return 2
    x = random.randint(2, n - 1)
    y = x
    c = random.randint(1, n - 1)
    d = 1
    iters = 0
    while d == 1 and iters < max_iter:
        x = (x * x + c) % n
        y = (y * y + c) % n
        y = (y * y + c) % n
        d = gcd(abs(x - y), n)
        iters += 1
    return d if d != n else None

# ============================================================================
#     FACTORIZACIÓN RECURSIVA TOTAL
# ============================================================================

def get_prime_factors(n):
    """Retorna lista de factores primos."""
    if n == 1: return []
    if is_prime(n): return [n]

    # Probar factores pequeños
    for p in [2,3,5,7,11,13,17,19,23,29,31,37,41,43,47]:
        if n % p == 0:
            return [p] + get_prime_factors(n // p)

    # Probar Pollard Rho
    factor = pollard_rho(n)
    if factor:
        return get_prime_factors(factor) + get_prime_factors(n // factor)

    # Si falla, retornar n (como compuesto desconocido o pseudo-primo)
    return [n]

# ============================================================================
#     EJECUCIÓN
# ============================================================================

def main():
    print(f"\n{'='*70}")
    print(f"🐉 INTEGER FACTORIZATION: DESCOMPOSICIÓN COMPLETA")
    print(f"{'='*70}")

    # Tu número N
    N = 25195908475657893494027183240048398571429282126204032027777137836043662020707595556264018525880784406918290641249515082189298559149176184502808489120072844992687392807287776735971418347270261896375014971824691165077613379859095700097330459748808428401797429100642458691817195118746121515172654632282216869987549182422433637259085141865462043576798423387184774447920739934236584823824281198163815010674810451660377306056201619676256133844143603833904414952634432190114657544454178424020924616515723350778707749817125772467962926386356373

    print(f"N = {str(N)[:50]}... ({N.bit_length()} bits)")

    # 1. Factores ya conocidos
    known_factors = [3, 3, 13171439]

    # 2. Calcular b (la parte restante)
    product_known = 1
    for f in known_factors: product_known *= f

    b = N // product_known
    print(f"\nFactores conocidos: {known_factors}")
    print(f"Resto b ({b.bit_length()} bits) = {str(b)[:50]}...")

    # 3. Factorizar b
    print(f"\n🔥 Factorizando b...")
    start = time.time()

    # Verificamos si b es primo primero (ahorra tiempo)
    if is_prime(b):
        factors_b = [b]
        print(f"  ✅ ¡b es PRIMO!")
    else:
        print(f"  ⚠️ b es COMPUESTO, descomponiendo...")
        factors_b = get_prime_factors(b)

    # 4. Resultado final
    all_factors = sorted(known_factors + factors_b)

    print(f"\n{'='*70}")
    print(f"🏆 FACTORIZACIÓN PRIMA COMPLETA 🏆")
    print(f"{'='*70}")

    print(f"\nN = ", end="")
    print(" × ".join(str(f) for f in all_factors))

    print(f"\nDetalle:")
    for f in all_factors:
        status = "✅ PRIMO" if is_prime(f) else "❌ COMPUESTO"
        print(f"  - {str(f)[:40]}{'...' if len(str(f))>40 else ''} ({f.bit_length()} bits) {status}")

    # Verificación final
    prod = 1
    for f in all_factors: prod *= f
    print(f"\nVerificación N = Π(factors): {prod == N}")
    print(f"Tiempo total: {time.time()-start:.2f}s")

if __name__ == "__main__":
    main()


🐉 INTEGER FACTORIZATION: DESCOMPOSICIÓN COMPLETA
N = 25195908475657893494027183240048398571429282126204... (1779 bits)

Factores conocidos: [3, 3, 13171439]
Resto b (1752 bits) = 21254666146836426818855878862040813014203840873004...

🔥 Factorizando b...
  ⚠️ b es COMPUESTO, descomponiendo...


KeyboardInterrupt: 

In [28]:

    print(f"  Factor 2 = {str(b)}")

  Factor 2 = 212546661468364268188558788620408130142038408730047829058828962643621568191832811351760751475478153120368081955792673679848737350474564991239407302422164646404722899190250264108672462078915699478200226108971179273895529898349634472127578971345233580373728245559218942438991544244129045810346450986210195408308581607880114586139205711737866586236733919228860554078164798970092998813774099466815323423784265769293842752878672808442706423295836492495487268177037639210741915345554190092845964468319363424474794370667605300866543523


 Factor 1 = 118542951